In [3]:
import pandas as pd
import plotly.express as px

df_car = pd.read_csv('../vehicles_us.csv')
print(df_car.sample(5))
df_car.info()

       price  model_year                  model  condition  cylinders fuel  \
46945  21900      2015.0          toyota tacoma  excellent        6.0  gas   
47979   3750      2007.0  chevrolet trailblazer  excellent        6.0  gas   
32616   5295      2013.0          ford focus se       good        4.0  gas   
3384   14400      2005.0          toyota tacoma  excellent        6.0  gas   
40469  11900         NaN         toyota corolla  excellent        4.0  gas   

       odometer transmission    type paint_color  is_4wd date_posted  \
46945   55000.0    automatic  pickup        grey     NaN  2018-09-03   
47979  141562.0    automatic     SUV        grey     1.0  2018-11-09   
32616   99385.0    automatic   sedan        grey     NaN  2018-06-01   
3384    79556.0    automatic  pickup      silver     1.0  2018-07-13   
40469   78948.0    automatic   sedan      silver     NaN  2018-06-09   

       days_listed  
46945           45  
47979           29  
32616            9  
3384          

In [ ]:
# Limpeza, validação dos dados e criação de colunas adicionais

# Transformação da coluna 'date_posted' para data e criação das colunas de ano e mês da postagem
df_car['date_posted'] = pd.to_datetime(df_car['date_posted'])
df_car['post_year'] = df_car['date_posted'].dt.year
df_car['post_month'] = df_car['date_posted'].dt.month

# Prenchimento do 'model_year' com a mediana usando transform mas matendo o tamanho original do df e alteração para o tipo int
df_car['model_year'] = df_car['model_year'].fillna(
    df_car.groupby('model')['model_year'].transform('median')
)
df_car['model_year'] = df_car['model_year'].astype(int)

# Prenchimento do 'odometer' com a mediana
df_car['odometer'] = df_car['odometer'].fillna(
    df_car.groupby('model_year')['odometer'].transform('median')
)

# Calculo da moda da variável 'cylinder' por modelo e preenchimento dos dados
cylinders_mode = (
    df_car.groupby('model')['cylinders'] # Agrupamos os dados pelo modelo
      .agg(pd.Series.mode) # calculamos a moda de cada.
      .reset_index() # Índice 'model' vira coluna
      .explode('cylinders') # Caso a moda tenha mais que um valor, elá se transforma em várias linhas
      .drop_duplicates('model') # Se exister múltiplas modas, ficamos só com a primeira
      .set_index('model')['cylinders'] # Transforma em uma Series de mapeamento
)

df_car['cylinders'] = df_car['cylinders'].fillna( # Preence somente os NaN da coluna original 'cylinders', nos lugares em que a moda é conhecida
    df_car['model'].map(cylinders_mode) # Para cada linha do df olha o model e busca o valor correspondete na Series
).astype(int)


# Prenchimento do 'paint_color' com a 'unknown'
df_car['paint_color'] = df_car['paint_color'].fillna('unknown')

# Prenchimento do 'is_4wd' com a 0 e transforamção para int
df_car['is_4wd'] = df_car['is_4wd'].fillna(0).astype(int)

# Removendo duplicados
df_car = df_car.drop_duplicates()

print(df_car.sample(5))
df_car.info()

         price  model_year                  model  condition  cylinders  \
17567   4495.0        2008       volkswagen jetta   like new          5   
40098   5995.0        2014           nissan versa       good          4   
1870   11800.0        2012             honda cr-v  excellent          4   
48202  32991.0        2008   ford f250 super duty   like new          8   
4148    4995.0        2008  chevrolet trailblazer       good          6   

         fuel  odometer transmission   type paint_color  is_4wd date_posted  \
17567     gas  119242.0    automatic  sedan        blue       0  2018-09-04   
40098     gas   82319.0    automatic  sedan        grey       0  2019-02-19   
1870      gas   98800.0    automatic    SUV       green       1  2018-08-26   
48202  diesel  122567.0    automatic  truck       black       1  2018-07-08   
4148      gas  158980.0    automatic    SUV       brown       1  2018-06-05   

       days_listed  post_year  post_month  
17567            7       2018 

In [ ]:
df_car = pd.read_csv('../vehicles_us.csv') # lendo os dados
fig = px.histogram(df_car, x="odometer") # criar um histograma
fig.show() # exibindo

In [ ]:
df_car = pd.read_csv('../vehicles_us.csv') # lendo os dados
fig = px.scatter(df_car, x="odometer", y="price") # criar um gráfico de dispersão
fig.show() # exibindo

In [9]:

fig1 = px.scatter(
    df_car,
    x='model_year',
    y='price',
    color='condition',
    title='Preço dos veículos por ano do modelo',
    labels={
        'model_year': 'Ano do modelo',
        'price': 'Preço',
        'condition': 'Condição'
    },
    opacity=0.6
)

fig1.show()

In [ ]:
df_price_type = (
    df_car.groupby('type', as_index=False)['price']
      .mean()
      .sort_values('price', ascending=False)
)

fig2 = px.bar(
    df_price_type,
    x='type',
    y='price',
    color='type',
    title='Preço médio por tipo de veículo',
    labels={
        'type': 'Tipo de veículo',
        'price': 'Preço médio'
    },
    text_auto='.2s',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig2.update_layout(
    showlegend=False,
    yaxis_tickprefix='$',
    template='plotly_white'
)

fig2.show()
